In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

import statistics
import random

from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel

from imblearn.over_sampling import SMOTE

from sklearn import metrics
from sklearn.metrics import fbeta_score

import statsmodels.api as sm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

In [ ]:
q1features_train_2 = pd.read_pickle("q1features_train_2.pickle")
q1features_valid_2 = pd.read_pickle("q1features_valid_2.pickle")
q1target_train_2 = pd.read_pickle("q1target_train_2.pickle")
q1target_valid_2 = pd.read_pickle("q1target_valid_2.pickle")

Imputation of missing data was performed. The alternative was to use complete case analysis, which involves removing from the dataset any rows that were missing any attributes. Complete case analysis leads to a loss of information and can lead to bias if the missing data is not missing completely at random.

Single-imputation methods were used; missing categorical values were imputed using the mode of that attribute and missing numeric values were imputed using the median. The validation data set was imputed using the mode and median values of the training data set. To avoid data leakage, the training data set was not imputed using validation set data. Data leakage can lead to inaccurately high model performance (but comparatively poor performance on the test data).

In [ ]:
#Imputing validation set
for col in q1features_valid_2.columns.tolist():
  q1features_valid_2[col] = q1features_valid_2[col].replace([None], np.nan)
  if (q1features_valid_2[col].dtype == 'category' or q1features_valid_2[col].dtype =='datetime64[ns]'): #
    q1features_valid_2[col]= q1features_valid_2[col].fillna(random.choice(statistics.multimode(q1features_train_2[col])))
  if (q1features_valid_2[col].dtype == 'Int64' or q1features_valid_2[col].dtype == 'int64'):
    q1features_valid_2[col] = q1features_valid_2[col].astype('float64')
  if (q1features_valid_2[col].dtype == 'float64'):
    q1features_valid_2[col] = q1features_valid_2[col].fillna(q1features_train_2[col].median())

In [ ]:
#Imputing training set
for col in q1features_train_2.columns.tolist():
  q1features_train_2[col] = q1features_train_2[col].replace([None], np.nan)
  if (q1features_train_2[col].dtype == 'category' or q1features_train_2[col].dtype =='datetime64[ns]'): #
    q1features_train_2[col] = q1features_train_2[col].fillna(random.choice(statistics.multimode(q1features_train_2[col])))
  if (q1features_train_2[col].dtype == 'Int64' or q1features_train_2[col].dtype == 'int64'):
    q1features_train_2[col] = q1features_train_2[col].astype('float64')
  if (q1features_train_2[col].dtype == 'float64'):
    q1features_train_2[col] = q1features_train_2[col].fillna(q1features_train_2[col].median())

Numeric attributes were standardized; variable importance will be assessed using regression models, and standardization allows for easier interpretation of the model coefficients. Standardization was performed instead of normalization, as standardization is more resistant to outliers.

In [ ]:
#Standardization of validation set
for col in q1features_valid_2.columns.tolist():
  if (q1features_valid_2[col].dtype == 'Int64' or q1features_valid_2[col].dtype == 'int64' or q1features_valid_2[col].dtype == 'float64'):
    for i in range(len(q1features_valid_2[col])):
      q1features_valid_2.loc[q1features_valid_2.index[i], col] = (q1features_valid_2.loc[q1features_valid_2.index[i], col] - q1features_train_2[col].mean())/np.std(q1features_train_2[col])

In [ ]:
#Standardization  of training set
for col in q1features_train_2.columns.tolist():
  if (q1features_train_2[col].dtype == 'Int64' or q1features_train_2[col].dtype == 'int64' or q1features_train_2[col].dtype == 'float64'):
    for i in range(len(q1features_train_2[col])):
      q1features_train_2.loc[q1features_train_2.index[i], col] = (q1features_train_2.loc[q1features_train_2.index[i], col] - q1features_train_2[col].mean())/np.std(q1features_train_2[col])

In [ ]:
#The SMOTE algorithm cannot handle datetime, so converting to number of
#days since January 1st, year one
for i in range(len(q1features_train_2['INTERVIEWDATE'])):
  q1features_train_2.loc[q1features_train_2.index[i], 'INTERVIEWDATE'] =  q1features_train_2.loc[q1features_train_2.index[i], 'INTERVIEWDATE'].toordinal()

for i in range(len(q1features_valid_2['INTERVIEWDATE'])):
  q1features_valid_2.loc[q1features_valid_2.index[i], 'INTERVIEWDATE'] =  q1features_valid_2.loc[q1features_valid_2.index[i], 'INTERVIEWDATE'].toordinal()

q1features_train_2['INTERVIEWDATE'] = q1features_train_2['INTERVIEWDATE'].astype('float64')
q1features_valid_2['INTERVIEWDATE'] = q1features_valid_2['INTERVIEWDATE'].astype('float64')

In [ ]:
#Converting categorical variables to dummy variables
for col in q1features_train_2.columns.tolist():
  if (q1features_train_2[col].dtype == 'category'):
    q1features_train_2 = pd.concat([q1features_train_2, pd.get_dummies(q1features_train_2[col], prefix=col, drop_first=True, dtype='float')], axis=1)
    q1features_train_2 = q1features_train_2.drop([col], axis=1)

In [ ]:
for col in q1features_valid_2.columns.tolist():
  if (q1features_valid_2[col].dtype == 'category'):
    q1features_valid_2 = pd.concat([q1features_valid_2, pd.get_dummies(q1features_valid_2[col], prefix=col, drop_first=True, dtype='float')], axis=1)
    q1features_valid_2 = q1features_valid_2.drop([col], axis=1)

In [ ]:
q1target_train_2 = q1target_train_2.astype('bool')
q1target_train_2 = q1target_train_2.astype('float')

In [ ]:
q1target_valid_2 = q1target_valid_2.astype('bool')
q1target_valid_2 = q1target_valid_2.astype('float')

 Multicollinearity can lead to model overfitting and can artificially hide the importance of explanatory variables. Multicollinearity (and collinearity) can be assessed using variance inflation factor values. A VIF value of over 10 indicates serious multicollinearity (Vittinghoff, E., Shiboski, S., Glidden, D., & McCulloch, C. (2004). Regression Methods in Biostatistics: Linear, Logistic, Survival and Repeated Measures Models. New York:Springer. https://doi.org/10.1007/b138825). Independent variables were iteratively removed from the model starting with the variable with the highest VIF value until all VIF values were below an accceptable threshold.

In [ ]:
#Removing highly correlated variables
vifcalcs=q1features_train_2

In [ ]:
vif_data = pd.DataFrame()
vif_data["feature"] = vifcalcs.columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(vifcalcs.values, i)
                          for i in range(len(vifcalcs.columns))]

In [ ]:
for i in range (len(vif_data['VIF'])):
  if (vif_data.loc[vif_data.index[i], 'VIF'])> 10:
    print(vif_data.loc[vif_data.index[i], 'feature'],vif_data.loc[vif_data.index[i], 'VIF'])

In [ ]:
#unwanted = vifcalcs.columns[vifcalcs.columns.str.startswith('PRIMINSR')]
#vifcalcs.drop(unwanted, axis=1, inplace=True)

In [ ]:
unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('IDAY')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('QSTVER')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('STATE')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('_AGE65YR')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('IMONTH')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('INTERVIEWDATE')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('_AGE_G')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('_AGE80')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('HTM4')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('WTKG3')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('USENOW3')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('EDUCA')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('SLEPTIM1')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('PRIMINSR')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('FMONTH')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_train_2.columns[q1features_train_2.columns.str.startswith('SMOKE100')]
q1features_train_2.drop(unwanted, axis=1, inplace=True)

In [ ]:
unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('IDAY')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('QSTVER')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('STATE')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('_AGE65YR')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('IMONTH')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('INTERVIEWDATE')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('_AGE_G')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('_AGE80')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('HTM4')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('WTKG3')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('USENOW3')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('EDUCA')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('SLEPTIM1')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('PRIMINSR')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('FMONTH')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_valid_2.columns[q1features_valid_2.columns.str.startswith('SMOKE100')]
q1features_valid_2.drop(unwanted, axis=1, inplace=True)

Feature selection can help reduce model overfitting. Feature selection can also result in dimensionality reduction, which can improve computational efficiency. Here, a decision tree model was used to evaluate feature importance; this model has a built-in method to compute Gini importance values for each attribute. Gini importance is also referred to as "mean decrease impurity" and is a measure of how a given attribute improves the purity of a node. Attributes with a Gini importance value of less than a robust threshold of 0.01 were removed from the model. Note that dummy variables generated using one-hot encoding will be grouped together; if one variable meets the threshold then all of the set of dummy variables will remain in the model. This is because the entire set of dummy variables is required to represent one categorical variable.

In [ ]:
clf = DecisionTreeClassifier(max_depth=16, random_state=8)
clf.fit(q1features_train_2, q1target_train_2)
y_pred = clf.predict(q1features_valid_2)

importances = clf.feature_importances_
threshold = 0.01
selected_features = q1features_train_2.columns[importances > threshold]
selected_features.tolist()

In [ ]:
filter_col = [col for col in q1features_train_2 if col.startswith('CHILDREN') or col.startswith('CPDEM') or col.startswith('DISPCODE') or col.startswith('DROCD') or col.startswith('IYEAR')
or col.startswith('PHYSHLTH') or col.startswith('SEQNO') or col.startswith('BLIND') or col.startswith('CVDCRH') or col.startswith('HAVARTH') or col.startswith('LASTDEN') or col.startswith('MARITAL')
or col.startswith('SEXVAR') or col.startswith('TETANUS') or col.startswith('_AGEG5YR')]

In [ ]:
q1features_train_2 = q1features_train_2[filter_col]
q1features_valid_2 = q1features_valid_2[filter_col]

The dataset is imbalanced; a majority of respondents reported that they did not have COVID-19 symptoms that lasted longer than 3 months. Imbalanced data adversely affects model performance; if the model encounters few instances of the minority class then it will be unable to effectively learn from this class.

The SMOTE algorithm was used to oversample the minority class; this address the issue of imbalanced data.

In [ ]:
#SMOTE Algorithm
smo = SMOTE(random_state = 2, k_neighbors=10)
q1features_train_2_SM, q1target_train_2_SM = smo.fit_resample(q1features_train_2, q1target_train_2.ravel())

Grid search was used to optimize model hyperparameters; the models were run with each possible combination of hyperparameters across all five sliding windows. The values of these hyperparameters were taken from Dr. Aamna AlShehhi and colleague's paper 'Utilizing machine learning for survival analysis to identify risk factors for COVID-19 intensive care unit admission: A retrospective cohort study from the United Arab Emirates' (https://doi.org/10.1371/journal.pone.0291373).

The hyperparameters were selected to optimize the models' F2 scores, which is a metric that is the harmonic mean of both precision and recall, although recall is weighted heavier than precision. This metric was also used by Dr. Roman Kessler and colleagues in their study "Predictive Attributes for Developing Long COVID—A Study Using Machine Learning and Real-World Data from Primary Care Physicians in Germany" (https://doi.org/10.3390/jcm12103511). It is logical to also prioritize recall over precision in this data analytics project, as here also there are more severe consequences for false negatives than false positives.

Logistic Regression

In [ ]:
hyperparameter_score_list = []
threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for p in threshold_list:
  logReg = sm.Logit(q1target_train_2_SM.ravel(), q1features_train_2_SM,).fit()
  logRegPrediction = logReg.predict(q1features_valid_2)
  logRegPrediction = np.where(logRegPrediction > p, 1, 0)

  f2_score = fbeta_score(q1target_valid_2, logRegPrediction, beta=2)
  accuracy =  metrics.accuracy_score(q1target_valid_2, logRegPrediction)
  recall =  metrics.recall_score(q1target_valid_2, logRegPrediction)
  precision = metrics.precision_score(q1target_valid_2, logRegPrediction)
  hyperparameter_score_list.append((p, f2_score, accuracy, recall, precision))

logRegScores2 = pd.DataFrame(hyperparameter_score_list, columns =['Threshold', 'F2_Score', 'Accuracy', 'Recall', 'Precision'])

In [ ]:
logRegScores2.to_csv('logRegScores2.csv')

Naive Bayes

In [ ]:
hyperparameter_score_list = []
threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for p in threshold_list:
  model = GaussianNB()
  nbModel = model.fit(q1features_train_2_SM, q1target_train_2_SM)
  nbPrediction = (nbModel.predict_proba(q1features_valid_2)[:,1] >= p).astype(bool)

  f2_score = fbeta_score(q1target_valid_2, nbPrediction, beta=2)
  accuracy =  metrics.accuracy_score(q1target_valid_2, logRegPrediction)
  recall =  metrics.recall_score(q1target_valid_2, logRegPrediction)
  precision = metrics.precision_score(q1target_valid_2, logRegPrediction)
  hyperparameter_score_list.append((p, f2_score, accuracy, recall, precision))

nbScores2 = pd.DataFrame(hyperparameter_score_list, columns =['Threshold', 'F2_Score', 'Accuracy', 'Recall', 'Precision'])

In [ ]:
nbScores2.to_csv('nbScores2.csv')

Random Forest

In [ ]:
hyperparameter_score_list = []
n_estimators_list =[25, 50, 75, 100]
max_depth_list = [1, 2, 3, 4, 5, 10]
threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for n in n_estimators_list:
  for m in max_depth_list:
    for p in threshold_list:
      rf = RandomForestClassifier(n_estimators=n, max_depth=m, random_state=42)
      rf.fit(q1features_train_2_SM, q1target_train_2_SM)
      #rfPrediction = rf.predict(q1features_valid_2)
      rfPrediction = (rf.predict_proba(q1features_valid_2)[:,1] >= p).astype(bool)

      f2_score = fbeta_score(q1target_valid_2, nbPrediction, beta=2)
      accuracy =  metrics.accuracy_score(q1target_valid_2, logRegPrediction)
      recall =  metrics.recall_score(q1target_valid_2, logRegPrediction)
      precision = metrics.precision_score(q1target_valid_2, logRegPrediction)
      hyperparameter_score_list.append((n, m, p, f2_score, accuracy, recall, precision))

rfScores2 = pd.DataFrame(hyperparameter_score_list, columns =['n_Estimators', 'Max_depth', 'Threshold', 'F2_Score', 'Accuracy', 'Recall', 'Precision'])

In [ ]:
rfScores2.to_csv('rfScores2.csv')

Light GBM

In [ ]:
train_data = lgb.Dataset(q1features_train_2_SM, label=q1target_train_2_SM)
test_data = lgb.Dataset(q1features_valid_2, label=q1target_valid_2, reference=train_data)

hyperparameter_score_list = []
num_leaves_list = [7, 31]
reg_alpha_list = [0.1, 0.5]
reg_lambda_list = [0.1, 0.5]
min_data_list = [5, 30, 100]
threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


for n in num_leaves_list:
  for a in reg_alpha_list:
    for l in reg_lambda_list:
      for d in min_data_list:
        for p in threshold_list:
          params = {
              "num_leaves": n,
              "reg_alpha": a,
              "reg_lambda": l,
              "min_data_in_leaf": d,
              "objective": "binary",
              "metric": "binary_logloss",
              "learning_rate": 0.05,
              "force_row_wise": True,
              "bagging_fraction": 0.8,
              "feature_fraction": 0.8
              }
          num_round=500
          bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])
          y_pred = bst.predict(q1features_valid_2)
          y_pred_binary = (y_pred > p).astype(int)

          f2_score = fbeta_score(q1target_valid_2, y_pred_binary, beta=2)
          accuracy =  metrics.accuracy_score(q1target_valid_2, y_pred_binary)
          recall =  metrics.recall_score(q1target_valid_2, y_pred_binary)
          precision = metrics.precision_score(q1target_valid_2, y_pred_binary)

          hyperparameter_score_list.append((n, a, l, d, p, f2_score, accuracy, recall, precision))

bstScores2 = pd.DataFrame(hyperparameter_score_list, columns =['Num_Leaves', 'Reg_Alpha', 'Reg_Lambda', 'Min_Data', 'Threshold', 'F2_Score', 'Accuracy', 'Recall', 'Precision'])

In [ ]:
bstScores2.to_csv('bstScores2.csv')